#  <center> Taller  de Aprendizaje Automático </center>
##  <center> Taller 9: *Natural Language Processing* (NLP)  </center>

## Introducción

La siguiente actividad propone el abordaje de un problema de procesamiento de lenguaje natural (NLP) utilizando herramientas de *embedding* y modelos RNN. El conjunto de datos que se utilizará es IMDb, el cual corresponde a un problema de clasificación donde se tienen 50000 criticas de películas (35000 de *train* y 15000 de *test*), y se quiere estimar si éstas son críticas positivas (1) o negativas (0).

La propuesta consiste en entender y reproducir los pasos de la sección *Sentiment Analysis* para los datos **sin procesar**, agregando algunas variantes como mitigar el sobreajuste y entender la herramienta *embeddings*.

En este Taller también se introduce la biblioteca *Streamlit*, utilizada para desarrollar prototipos de aplicaciones web de aprendizaje automático. Aquellos que así lo deseen, podrán generar de manera sencilla una aplicación web que clasifique las críticas proporcionadas por los usuarios. Además, se presenta la biblioteca HuggingFace, la cual permite realizar inferencias con modelos preentrenados y realizar fine-tuning para adaptarlos a esta tarea específica.

## Objetivos


*   Aplicar modelos basados en RNN a un problema de NLP.
*   Trabajar con embeddings para secuencias de texto, en particular embeddings preentrenados.
*   Utilizar herramientas para la visualización de embeddings.
*  (Opcional, no evaluado) Desarrollar una aplicación web que clasifique críticas proporcionadas por los usarios
*  (Opcional, no evaluado) Utilizar la biblioteca *HuggingFace* para utilizar modelos preentrenados y realizar fine-tunning.

## Formas de trabajo

### Opción 1: Trabajar localmente

Descargar los datos en su máquina personal y trabajar en su propio ambiente de desarrollo.

`conda activate TAA-py310`              
`jupyter-notebook`    

Los paquetes faltantes se pueden instalar desde el notebook haciendo:     
` !pip install paquete_faltante`

### Opción 2:  Trabajar en *Colab*.

<table align="center">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/TAA-fing/TAA-2025/blob/main/talleres/taller9_NLP.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Ejecutar en Google Colab</a>
  </td>
</table>

Se puede trabajar en Google Colab. Para ello es necesario contar con una cuenta de **google drive** y ejecutar un notebook almacenado en dicha cuenta. De lo contrario, no se conservarán los cambios realizados en la sesión. En caso de ya contar con una cuenta, se puede abrir el notebook y luego ir a `Archivo-->Guardar una copia en drive`.

La siguiente celda realiza la configuración necesaria para obtener datos desde la plataforma Kaggle. Le solicitará que suba el archivo *kaggle.json* asociado a su cuenta.

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from google.colab import files

# El siguiente archivo solicitado es para habilitar la API de Kaggle en el entorno que está trabajando.
# Este archivo se descarga entrando a su perfíl de Kaggle, en la sección API, presionando donde dice: Create New API Token

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

#Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 64 bytes


# Parte 1: Análisis y preprocesamiento de datos


Se utilizará el conjunto de IMDb provisto por Kaggle. Se tienen 50000 criticas de películas que al igual que en el *Taller 2* se utilizarán 35000 para *train* y 15000 para *test*.

*   Ejecutar la siguiente celda para descargar el conjunto y verificar que los conjuntos tienen la cantidad de instancias esperadas.

In [ ]:
# Descarga la base IMDb de Kaggle
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
  0% 0.00/25.7M [00:00<?, ?B/s]
100% 25.7M/25.7M [00:00<00:00, 1.22GB/s]


In [ ]:
import zipfile
import pandas as pd
import tensorflow as tf

# Se descomprime el archivo descargado
with zipfile.ZipFile('imdb-dataset-of-50k-movie-reviews.zip', 'r') as zip_ref:
    zip_ref.extractall('')

# Se levanta como pandas DataFrame
data_file = 'IMDB Dataset.csv'
data = pd.read_csv(data_file)

#Separación de Conjuntos
N=35000
X_train = data.loc[:N-1, 'review'].values
y_train = data.loc[:N-1, 'sentiment'].values == 'positive'
X_test = data.loc[N:, 'review'].values
y_test = data.loc[N:, 'sentiment'].values == 'positive'

# Armado de los Tensorflow Datasets
data_train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
data_test = tf.data.Dataset.from_tensor_slices((X_test, y_test))

#Verificación
print('Train Size:', len(data_train),'Test Size:', len(data_test))

Train Size: 35000 Test Size: 15000


*   Del conjunto de entrenamiento visualizar tanto una review positiva como una negativa. Se sugiere ir a los Notebooks del *Capítulo 16*. Puede ser útil el uso del método *.skip()*.

In [ ]:
for review, label in data_train.take(4):
    print(review.numpy().decode("utf-8")[:200], "...")
    print("Label:", label.numpy())

One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me abo ...
Label: True
A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece ...
Label: True
I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is wi ...
Label: True
Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, J ...
Label: False


*   Reservar unas 5000 críticas de los datos de entrenamiento para validación.

In [ ]:
train_size = len(data_train)

val_size = 5000

data_val = data_train.skip(train_size - val_size)
data_train = data_train.take(train_size - val_size)

print('Tamaño de train: ', len(data_train))
print('Tamaño de val: ', len(data_val))

Tamaño de train:  30000
Tamaño de val:  5000


* Prepare los batches para todos los conjuntos

In [ ]:
train_set =data_train.shuffle(5000, seed=42).batch(32).prefetch(1)
valid_set = data_val.batch(32).prefetch(1)
test_set = data_test.batch(32).prefetch(1)

*   Keras proporciona una capa de `TextVectorization` para el preprocesamiento básico de texto. Explique el funcionamiento y adapte una capa para los datos de entrenemiento en cuestión. Se sugiere leer la sección asociada a esta capa en el *Capítulo 13* del libro. Utilice un tamaño de vocabulario de 10000 palabras.


In [ ]:
vocab_size = 10000
text_vec_layer = tf.keras.layers.TextVectorization(max_tokens=vocab_size)
text_vec_layer.adapt(train_set.map(lambda reviews, labels: reviews))

* Obtenga el diccionario de la capa `TextVectorization`

In [ ]:
dictionary = text_vec_layer.get_vocabulary()
for numero, valor in enumerate(dictionary[:20]):
  print(f'{numero}: {valor}')

0: 
1: [UNK]
2: the
3: and
4: a
5: of
6: to
7: is
8: in
9: it
10: i
11: this
12: that
13: br
14: was
15: as
16: with
17: for
18: movie
19: but


* Siguiendo el ejemplo adjunto en la [documentación](https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization) de la capa `TextVectorization`. Cree un modelo de keras que cuente únicamente con esta capa y observe el resultado de pasar una crítica cualquiera.

In [ ]:
text_vec_model = tf.keras.Sequential([
    text_vec_layer
    ])


pred = text_vec_model.predict(train_set.take(1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step


In [ ]:
for reviews_batch, _ in train_set.take(1):
    review_text = reviews_batch[0].numpy().decode("utf-8")
    print("Texto original:")
    print(review_text[:100], "...")
    vectorized_batch = text_vec_layer(reviews_batch)
    print("\nTexto vectorizado:")
    print(vectorized_batch[0].numpy())


Texto original:
**Warning! Mild Spoilers Ahead!**<br /><br />(Yes, I realize it's tough to spoil an historical docum ...

Texto vectorizado:
[1727 3616 1200    1   13  417   10  912   30 1263    6 2305   34 1269
  673   19   10   79 2460   47    5    2 7197    3    1   13   11    7
   34 3068  673   22   41   80    5    2 1832  895   19   82  660    6
    2   66  513    9   80    2    1  112   22  289   46    6  371    2
   66    5 4193   19  242   12    5    4 8745    1    2 7072 1302    3
    2  728 1821   16   95   24   53  146    3  984   71   35   57   27
    8    4 1283    1   13   13    8    4 1485  283 4193    7 8872   44
    2  710   68   34  761  232   35   57   27  905   15    2  102   24
 3002   91 1340    4    1    6   61   35  208 4444   11    7   32    2
   53  495 1052    2 1463    3 1368    5    2 2727  895   12    2  677
   69    6    1  142    6  407   11   66  290   13   13    2   89 4579
    5    2   20 1525    4 3197    5  121  928    4  965 1602 3858   86
    4 8

*   Utilizando esta capa ¿cuáles son los largos de secuencias para los primeros *batches*? ¿Es un problema a resolver que todos tengan largos distintos? ¿Por qué?.

In [ ]:
for reviews_batch, _ in train_set.take(1):
    vectorized = text_vec_layer(reviews_batch)
    print("Forma del batch vectorizado:", vectorized.shape)
    print("Batch vectorizado:", vectorized[:1])


Forma del batch vectorizado: (32, 960)
Batch vectorizado: tf.Tensor(
[[  44   23 1095  697   40   67  263   58   16 2125   23   78   63  352
    11   18   44   21    2   77  587   23   41  181    6   65 3095   87
   303 2086   11    7    4  208   65   22   34 9198    5    1    8 1737
     1  754 1779  122  527   46 7675    1  618    7    2    1    5 3404
     3  264   10   69  120    1   43   42  172    7   55    8  603   55
     2  190   12   59   14 2814   16  299 7403  112   22 6418   17   42
   892 7675    7    4  718   44  120   48   14   29    1    1    3    1
     1   24    1  452 1201    9    7    4  177   50   66  425  113 6666
    24 6930  290    8    4  427    5 1563 8068    5 9712  433    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0 

In [ ]:
for reviews_batch, labels_batch in train_set.take(1):
    for review in reviews_batch[:1]:
        print(review.numpy().decode("utf-8")[:200], "...")

The Assignment is an outstanding thriller with several plot twists driven by character, rather than star turns, the need to stage special effects, obligatory romance, and endless car chases. However,  ...


Las capas esperan entradas de tama;o fijo por batch
Durante el .adapt(...), si no se define explícitamente output_sequence_length, la capa estima una longitud adecuada con base en la distribución de longitudes del texto del dataset

# Parte 2: *Embedding*

* Cree y entrene el modelo que aparece al final de la sección *Sentiment Analysis* y previo a la sección *Masking* del *Capítulo 16*.

In [ ]:
embed_size = 128
tf.random.set_seed(42)
model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(vocab_size, embed_size),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=2)

Epoch 1/2
938/938 ━━━━━━━━━━━━━━━━━━━━ 44s 37ms/step - accuracy: 0.5091 - loss: 0.6936 - val_accuracy: 0.4960 - val_loss: 0.6925
Epoch 2/2
938/938 ━━━━━━━━━━━━━━━━━━━━ 40s 39ms/step - accuracy: 0.5123 - loss: 0.6916 - val_accuracy: 0.4958 - val_loss: 0.6935


* Implemente el uso de *Masking* en el modelo que ya utilizó. ¿Por qué podría ser esto útil para el aprendizaje?

In [ ]:
embed_size = 128
tf.random.set_seed(42)
model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(vocab_size, embed_size, mask_zero=True),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=5)

Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 40s 40ms/step - accuracy: 0.7001 - loss: 0.5538 - val_accuracy: 0.8406 - val_loss: 0.3842
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 37s 36ms/step - accuracy: 0.8798 - loss: 0.2990 - val_accuracy: 0.8812 - val_loss: 0.2899
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 43s 38ms/step - accuracy: 0.9330 - loss: 0.1782 - val_accuracy: 0.9008 - val_loss: 0.2539
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 38s 35ms/step - accuracy: 0.9641 - loss: 0.1070 - val_accuracy: 0.8850 - val_loss: 0.3351
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 33s 35ms/step - accuracy: 0.9776 - loss: 0.0684 - val_accuracy: 0.8838 - val_loss: 0.3855


* Observe que el modelo se sobreajusta a los datos. Utilice alguna técnica vista para regularizar.

In [ ]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization              │ (None, None)           │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, None, 128)      │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 128)            │        99,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,137,606 (15.78 MB)

 Trainable params: 1,379,201 (5.26 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,758,405 (10.52 MB)

In [ ]:
model_dropout = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(vocab_size, embed_size, mask_zero=True),
    tf.keras.layers.GRU(128, dropout=0.3),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model_dropout.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(
    patience=4,  # esperar 4 épocas sin mejora
    restore_best_weights=True
)

history = model_dropout.fit(
    train_set,
    validation_data=valid_set,
    epochs=10,
    callbacks=[early_stop]
)

Epoch 1/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 42s 42ms/step - accuracy: 0.6907 - loss: 0.5569 - val_accuracy: 0.8768 - val_loss: 0.3110
Epoch 2/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 48s 49ms/step - accuracy: 0.9040 - loss: 0.2426 - val_accuracy: 0.8948 - val_loss: 0.2637
Epoch 3/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 81s 49ms/step - accuracy: 0.9417 - loss: 0.1607 - val_accuracy: 0.8880 - val_loss: 0.2954
Epoch 4/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 53s 56ms/step - accuracy: 0.9621 - loss: 0.1067 - val_accuracy: 0.8706 - val_loss: 0.4106
Epoch 5/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 35s 38ms/step - accuracy: 0.9757 - loss: 0.0737 - val_accuracy: 0.8822 - val_loss: 0.4093
Epoch 6/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 38s 35ms/step - accuracy: 0.9841 - loss: 0.0486 - val_accuracy: 0.8752 - val_loss: 0.4931


In [ ]:
from tensorflow.keras import regularizers


model_dropout3 = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(vocab_size, embed_size, mask_zero=True),
    tf.keras.layers.LSTM(128, dropout=0.3,return_sequences=True,kernel_regularizer=regularizers.l2(0.00001)),
    tf.keras.layers.LSTM(128, dropout=0.5,kernel_regularizer=regularizers.l2(0.00001)),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model_dropout.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])


model_dropout3.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(
    patience=4,  # esperar 4 épocas sin mejora
    restore_best_weights=True
)

history = model_dropout3.fit(
    train_set,
    validation_data=valid_set,
    epochs=10,
    callbacks=[early_stop])

Epoch 1/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 65s 63ms/step - accuracy: 0.6950 - loss: 0.5668 - val_accuracy: 0.8512 - val_loss: 0.3512
Epoch 2/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 59s 63ms/step - accuracy: 0.8777 - loss: 0.3095 - val_accuracy: 0.8582 - val_loss: 0.3339
Epoch 3/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 59s 63ms/step - accuracy: 0.8955 - loss: 0.2752 - val_accuracy: 0.8756 - val_loss: 0.3276
Epoch 4/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 84s 65ms/step - accuracy: 0.8753 - loss: 0.2926 - val_accuracy: 0.7086 - val_loss: 0.5686
Epoch 5/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 61s 65ms/step - accuracy: 0.7451 - loss: 0.4922 - val_accuracy: 0.8558 - val_loss: 0.3750
Epoch 6/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 63s 67ms/step - accuracy: 0.9085 - loss: 0.2554 - val_accuracy: 0.8722 - val_loss: 0.3315
Epoch 7/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 90s 96ms/step - accuracy: 0.9382 - loss: 0.1874 - val_accuracy: 0.8688 - val_loss: 0.3713


In [ ]:
embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size, mask_zero=True)

In [ ]:
from comet_ml import Experiment
API_KEY = 'B1BQZyV4ItS1xcnh2qC0PDToK'

ModuleNotFoundError: No module named 'comet_ml'

In [ ]:
exp = Experiment(api_key=API_KEY,
                 project_name='Taller9',
                 auto_param_logging=False)

In [ ]:
embeddings = model.layers[1].get_weights()[0]

dictionary[0] = "<pad>"


exp.log_embedding(vectors= embeddings , labels= dictionary)

* El modelo cuenta con una capa de entrada de *embedding* la cual abarca la mayoría de los parámetros entrenables. En este caso un *embedding* es un vector entrenable que representa una palabra en nuevo espacio cuyo tamaño es un hiperparámetro. La concatenación de estos vectores conforma la matriz de *embedding*, donde su cantidad de filas corresponde a la suma del tamaño del vocabulario y la cantidad de columnas a las dimensiones de los vectores (*embed_size*). Al igual que una matriz de pesos, ésta se inicializa de forma aleatoria, y actualiza sus valores para cada *step* de entrenamiento.

  *   ¿Cuál es la ventaja de utilizar una capa de *embedding*? (Ver la sección *Encoding Categorical Features Using Embeddings* del *Capítulo 13* del libro.)
  *   Visualizar una representación del espacio de *embedding* utilizando *Comet*. **Importante:** para evitar errores, modifique del diccionario el valor del espacio por `<pad>`.

  Para este último punto se recomienda seguir el siguiente ejemplo: [logging-embeddings](https://www.comet.ml/docs/user-interface/embeddings/#logging-embeddings).

*   Para su mejor modelo: visualizar a qué distancias se encuentran las palabras unas de otras tanto en la representación a baja dimensión como en el espacio de *embedding*. Sobre todo probar con adjetivos positivos (*wonderful*, *excellent*, etc.) y negativos (*ugly*, *boring*, etc.) comparando los resultados. ¿Qué logra observar?.

# Parte 3: *Embedding Preentrenado*

Una de las técnicas para mejorar el desempeño en este tipo de problemas es utilizar *embeddings* ya entrenados.
Siguiendo el ejemplo [Using pre-trained word embeddings](https://keras.io/examples/nlp/pretrained_word_embeddings/):

*   Descargar el *embedding* preentrenado [GloVE](https://nlp.stanford.edu/projects/glove/) que aparece en la sección *Load pre-trained word embeddings*.

In [ ]:
!wget --no-check-certificate https://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2025-06-28 17:12:54--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-06-28 17:12:54--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.68MB/s    in 2m 42s  

2025-06-28 17:15:37 (5.08 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



* Preparar la nueva matriz de *embedding*. ¿Cuántas palabras del conjunto de entrenamiento se encuentran en el vocabulario de GloVE? ¿Cuántas no?.

In [ ]:
import os
import numpy as np

path_to_glove_file = '/content/glove.6B.100d.txt'

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [ ]:
word_index = dict(zip(dictionary, range(len(dictionary))))

In [ ]:
num_tokens = len(dictionary)
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 9699 words (301 misses)


* Entrenar el modelo con la nueva matriz de *embedding* de manera que los valores de ésta se mantengan fijos (ver parámetro en la capa de *embedding*). Utilice masking.

In [ ]:
from keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    trainable=False,
    mask_zero = True
)
embedding_layer.build((1,))
embedding_layer.set_weights([embedding_matrix])

In [ ]:
model_emb_not_trainable = tf.keras.Sequential([
    text_vec_layer,
    embedding_layer,
    tf.keras.layers.GRU(128, dropout = 0.4),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model_emb_not_trainable.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])
history = model_emb_not_trainable.fit(train_set, validation_data=valid_set, epochs=2)

Epoch 1/2
938/938 ━━━━━━━━━━━━━━━━━━━━ 40s 39ms/step - accuracy: 0.6543 - loss: 0.6034 - val_accuracy: 0.7958 - val_loss: 0.4357
Epoch 2/2
938/938 ━━━━━━━━━━━━━━━━━━━━ 34s 36ms/step - accuracy: 0.8263 - loss: 0.3886 - val_accuracy: 0.8372 - val_loss: 0.3592


* Entrene nuevamente el modelo pero con el *embedding* entrenable. Modifique el *learning rate*.

In [ ]:
embedding_layer_trainable = Embedding(
    num_tokens,
    embedding_dim,
    trainable=True,
    mask_zero = True
)
embedding_layer_trainable.build((1,))
embedding_layer_trainable.set_weights([embedding_matrix])


model_trainable = tf.keras.Sequential([
    text_vec_layer,
    embedding_layer_trainable,
    tf.keras.layers.GRU(128, dropout = 0.4),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model_trainable.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])
history = model_emb_not_trainable.fit(train_set, validation_data=valid_set, epochs=2)

Epoch 1/2
938/938 ━━━━━━━━━━━━━━━━━━━━ 35s 37ms/step - accuracy: 0.8511 - loss: 0.3402 - val_accuracy: 0.8498 - val_loss: 0.3386
Epoch 2/2
938/938 ━━━━━━━━━━━━━━━━━━━━ 40s 36ms/step - accuracy: 0.8626 - loss: 0.3167 - val_accuracy: 0.8594 - val_loss: 0.3247


In [ ]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization              │ (None, None)           │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, None, 128)      │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 128)            │        99,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,137,606 (15.78 MB)

 Trainable params: 1,379,201 (5.26 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,758,405 (10.52 MB)

In [ ]:
model_emb_not_trainable.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization              │ (None, None)           │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_4 (Embedding)         │ (None, None, 100)      │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ (None, 128)            │        88,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,265,350 (4.83 MB)

 Trainable params: 88,449 (345.50 KB)

 Non-trainable params: 1,000,000 (3.81 MB)

 Optimizer params: 176,901 (691.02 KB)

In [ ]:
model_trainable.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization              │ (None, None)           │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_5 (Embedding)         │ (1, 100)               │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_4 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,000,000 (3.81 MB)

 Trainable params: 1,000,000 (3.81 MB)

 Non-trainable params: 0 (0.00 B)

*   Comparar con los modelos anteriores en cuanto al desempeño, la cantidad de parámetros y el tiempo de entrenamiento.


In [ ]:


dictionary[0] = "<pad>"


exp.log_embedding(vectors= embedding_matrix , labels= dictionary)

Respuesta:

*   Visualizar cómo es el espacio de *embedding*. ¿Qué diferencias puede observar con respecto al anterior?


Respuesta:

# Parte 4: Mejoras en los Modelos

Se sugieren algunas líneas que podrían mejorar los desempeños obtenidos en las partes anteriores:

* Ampliar el tamaño del vocabulario utilizado durante el entrenamiento, junto con aumentar la complejidad del modelo.
* Implementar una función de preprocesamiento de texto que realice tareas como eliminar las etiquetas HTML, eliminar números y llevar a cabo otras acciones relevantes.
* Considerar la utilización de n-gramas y/o modificar el tokenizador utilizado.
* Evaluar el cambio de las neuronas recurrentes de GRU a LSTM.

Piense por qué tiene sentido explorar estas estrategias y pruebe alguna que considere relevante o proponga alguna otra que le parezca relevante.


In [ ]:
import re
import string

def clean_text(text):
    # Eliminar HTML
    text = re.sub(r'<.*?>', '', text)
    # Eliminar números
    text = re.sub(r'\d+', '', text)
    # Eliminar puntuación
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Lowercase
    text = text.lower()
    return text

In [ ]:
data_train_clean = [(clean_text(review.numpy().decode()), label.numpy()) for review, label in train_set.unbatch()]
data_valid_clean = [(clean_text(review.numpy().decode()), label.numpy()) for review, label in valid_set.unbatch()]
data_test_clean  = [(clean_text(review.numpy().decode()), label.numpy()) for review, label in test_set.unbatch()]


In [ ]:
import tensorflow as tf

data_train_clean_ds = tf.data.Dataset.from_tensor_slices(
    [x for x, y in data_train_clean]
)

In [ ]:
vocab_size2 = 20000
text_vec_layer = tf.keras.layers.TextVectorization(max_tokens=vocab_size2)
text_vec_layer.adapt(data_train_clean_ds)


In [ ]:
dictionary = text_vec_layer.get_vocabulary()
word_index = dict(zip(dictionary, range(len(dictionary))))
embedding_dim = embedding_matrix.shape[1]
vocab_size = embedding_matrix.shape[0]


num_tokens = len(dictionary) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))





embedding_layer_f = tf.keras.layers.Embedding(
    input_dim=num_tokens,
    output_dim=embedding_dim,
    weights=[embedding_matrix],
    trainable=True,
    mask_zero=True
)


Converted 19246 words (754 misses)


In [ ]:
embedding_layer_f.build((1,))
embedding_layer_f.set_weights([embedding_matrix])


model_trainable = tf.keras.Sequential([
    text_vec_layer,
    embedding_layer_f,
    tf.keras.layers.GRU(128, dropout = 0.4),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model_trainable.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])
history = model_emb_not_trainable.fit(train_set, validation_data=valid_set, epochs=10)

Epoch 1/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 33s 35ms/step - accuracy: 0.8714 - loss: 0.2937 - val_accuracy: 0.8648 - val_loss: 0.3135
Epoch 2/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 33s 35ms/step - accuracy: 0.8818 - loss: 0.2782 - val_accuracy: 0.8700 - val_loss: 0.3050
Epoch 3/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 41s 35ms/step - accuracy: 0.8868 - loss: 0.2665 - val_accuracy: 0.8700 - val_loss: 0.3079
Epoch 4/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 41s 36ms/step - accuracy: 0.8946 - loss: 0.2555 - val_accuracy: 0.8682 - val_loss: 0.3106
Epoch 5/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 34s 37ms/step - accuracy: 0.8918 - loss: 0.2531 - val_accuracy: 0.8814 - val_loss: 0.2876
Epoch 6/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 33s 35ms/step - accuracy: 0.9010 - loss: 0.2364 - val_accuracy: 0.8628 - val_loss: 0.3141
Epoch 7/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 33s 35ms/step - accuracy: 0.9025 - loss: 0.2292 - val_accuracy: 0.8764 - val_loss: 0.2953
Epoch 8/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 34s 37ms/step - accuracy: 0.9045 - loss: 0.2275 - 

In [ ]:
model_trainable.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])
texts_train, labels_train = zip(*data_train_clean)
texts_valid, labels_valid = zip(*data_valid_clean)

history = model_emb_not_trainable.fit(
    x=list(texts_train),
    y=np.array(labels_train),
    validation_data=(list(texts_valid), np.array(labels_valid)),
    epochs=10)

Buffered data was truncated after reaching the output size limit.

In [ ]:
for text_tensor in data_train_clean_ds.take(5):
    print(text_tensor.numpy().decode("utf-8"))

# Parte 5: Desarrollo de una aplicación web (Opcional)

En esta parte veremos cómo generar una aplicación web sencilla que permita mostrar el funcionamiento de un modelo que hayamos entrenado. Para ello utilizaremos la biblioteca [Streamlit](https://streamlit.io/).

## Streamlit
Streamlit es una biblioteca de código abierto escrita en Python que permite crear y compartir aplicaciones web que usan algoritmos de aprendizaje automático. En la [documentación](https://docs.streamlit.io/) encontrará información sobre cómo instalar y crear aplicaciones utilizando la biblioteca. El flujo de trabajo básico consta de los siguientes pasos:   

1. Instalación   
2. Desarrollo de la aplicación   
3. Desplieque de la aplicación

### Instalación
En la mayoría de los casos, la biblioteca debería quedar instalada luego de crear un ambiente virtual (por ejemplo de conda) y hacer:   

`pip install streamlit`  

Puede verificar que la instalación sea correcta haciendo:    

`streamlit hello`

Puede ver los detalles de instalación en https://docs.streamlit.io/library/get-started/installation

### Desarrollo de la aplicación  

Se sugiere desarrollar la aplicación partiendo de un ejemplo que clasifica críticas de cine utilizando un modelo entrenado con las técnicas vistas en el Taller 2. Dicho ejemplo puede verse en funcionamiento [acá](https://share.streamlit.io/taa-fing/taa-2022/main/apps/movie_review_app/movie_review_app.py). La siguiente celda descarga el código fuente y lo descomprime.

In [ ]:
!wget iie.fing.edu.uy/~carbajal/movie_review/apps.zip
!unzip apps.zip

Copie el archivo *movie_review_app.py*, modifique el nombre, y realice las siguientes modificaciones (además de las de diseño que crea conveniente):   

**Cambio de Modelo:**  Para hacer inferencia fuera de este *Notebook* será necesario contar con el modelo entrenado. El modelo se puede guardar con alguna de las técnicas vistas en el curso. Se sugiere utilizar el *Callback* **ModelCheckpoint**.

**Modificación del pipeline de inferencia:**  El objetivo es generar una función que a partir de una única reseña prediga si la reseña es *positiva*(1) o *negativa*(0). Para ello se brinda una función a completar

In [ ]:
# Pipeline funtion to make inference
def pipeline_inference (review, model):

  '''
  Función que prepara una review aislada y hace inferencia con el modelo.

  Entradas:
    review: String con la review a hacer inferencia
    model: Modelo entrenado con el que se realiza inferencia

  Salida:

    pred: Predicción del modelo

  '''

  # ...

  return pred

* Las siguientes celdas prueban la función. Verifique que funciona correctamente.

In [ ]:
# Load model
checkpoint_filepath = '/content/...'

model_loaded = keras.models.load_model(checkpoint_filepath)

In [ ]:
review = 'This movie is really boring. I do not recommend it.'

pred = pipeline_inference(review, model_loaded)

pred

In [ ]:
review = 'This movie is wonderful. I love it.'

pred = pipeline_inference(review, model_loaded)

pred

### Correr la aplicación localmente

Una vez realizadas las modificaciones en el archivo principal, cree un directorio donde guardar los archivos de su aplicación. Guarde allí el modelo, el archivo tipo numpy con las palabras y su archivo principal (Ej. *movie_review_app.py*).

* Una vez modificado el código, puede probarlo localmente. Para ello ejecute el siguiente comando, sustituyendo *movie_review_app.py* por el nombre de su archivo principal.


`!streamlit run apps/movie_review_app/movie_review_app.py`    

* Si en vez de localmente, está corriendo el notebook en Colab, ejecute:

`!streamlit run apps/movie_review_app/movie_review_app.py & npx localtunnel --port 80`      

### Despliegue de la aplicación

Una vez que la app fue desarrollada es posible compartirla para que otros puedan probarla. Para ello es necesario:  

1. Contar con una cuenta de [Streamlit Cloud](https://docs.streamlit.io/streamlit-cloud/get-started#sign-up-for-streamlit-cloud) y un repositorio de GitHub donde almacenar el código.
2.  Subir al repositorio  el código y los datos necesarios para correrlo.
3. [Conectar la cuenta de Streamlit Cloud con la del repositorio](https://docs.streamlit.io/streamlit-cloud/get-started#connect-your-github-account).     
4. [Publicar la app](https://docs.streamlit.io/streamlit-cloud/get-started/deploy-an-app)

*Comentario:* Puede que sea necesario agregar en el repositorio un archivo *requirements.txt* donde deba especificar las liberías utilizas en el archivo *main.py*.

# Parte 6: Uso de la liberería HuggingFace (Opcional)

Hugging Face es una organización de desarrolladores que utilizan una [plataforma](https://huggingface.co/) para compartir y desarrollar modelos de código abierto con arquitectura de Transformers, centrados principalmente en procesamiento de lenguaje natural, aunque también cuentan con modelos para procesamiento de imágenes y señales de audio.  

Los modelos son desarrollados con técnicas del estado del arte, y entrenados con grandes volumnes de datos y capacidad de computo. Cualquier persona que quiera hacer uso de los modelos pre-entrenados puede hacerlo simplemente instalando la biblioteca de Transformers y los módulos necesarios para las tareas a implementar.  

## Uso de modelo preentrenado para hacer inferencia

Este código utiliza la biblioteca de HuggingFace "transformers" para importar y utilizar un modelo preentrenado de análisis de sentimientos. En este caso, se utiliza el modelo "sentiment-analysis", que se encarga de clasificar el sentimiento de un texto dado como positivo o negativo.

In [ ]:
import sys

if "google.colab" in sys.modules:
    %pip install -q -U transformers
    %pip install -q -U datasets

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
result = classifier("The movie was horrible.")

In [ ]:
result

Utilizando este modelo realice inferencia sobre el conjunto de test, y evalue el desempeño obtenido. Compárelo con los obtenidos anteriormente.

In [ ]:
N_test = X_test.shape[0]

pred_test = np.zeros(N_test)

for i in range(N_test):
  result = classifier(X_test[i][:1500]) # Corta la review a 1500 caracteres para adaptarse al modelo
  if result[0]['label'] == 'NEGATIVE':
    pred_test[i] = 1 - result[0]['score']
  else:
    pred_test[i] = result[0]['score']

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, pred_test > 0.5)

## Fine-tuning de un modelo ya pre-entrenado


Para ejecutar esta parte del notebook, deberá reiniciar el entorno de ejecución.

La siguiente celda instala las bibliotecas necesarias para esta parte.

In [ ]:
%%capture
# Instalamos la biblioteca con los modelos de arquitectura Transformer
!pip install transformers
!pip install datasets
!pip install accelerate
!pip install torch

El objetivo será realizar un ajuste de parámetros al modelo utilizando los datos del problema de interés, por lo tanto la siguiente celda levanta nuevamente los datos de la críticas de cine IMBd.

In [ ]:
# Se levanta como pandas DataFrame (Se requiere tener el csv descargado de la página de Kaggle)
data_file = 'IMDB Dataset.csv'
data = pd.read_csv(data_file)

# Reduce el largo de las reviews a 1500 caracteres
data['review'] = data['review'].str[:1500]

# Convierte las etiquetas de sentimiento a 1 y 0
data['sentiment'] = data['sentiment'].map({'positive': 1, 'negative': 0})

# Print los datos actualizados
print(data.head())

In [ ]:
data.iloc[0]['review']

**Armado de Conjuntos** Se arman los datasets de Torch para realizar el entrenamiento.

In [ ]:
import datasets

# Convertir los pandas DataFrames a datasets
train_data = datasets.Dataset.from_pandas(data[:20000]) # Se cortan la cantidad de reseñas para reducir el costo computacional de la ejecucción.
val_data = datasets.Dataset.from_pandas(data[30000:35000])
test_data = datasets.Dataset.from_pandas(data[35000-1:])

# Renombrar las columnas para que coincidan con los nombres esperados por el modelo
train_data = train_data.rename_column("sentiment", "label")
val_data = val_data.rename_column("sentiment", "label")
test_data = test_data.rename_column("sentiment", "label")
train_data = train_data.rename_column("review", "text")
val_data = val_data.rename_column("review", "text")
test_data = test_data.rename_column("review", "text")

# Crear un DatasetDict
emotions = datasets.DatasetDict({
    "train": train_data,
    "validation": val_data,
    "test": test_data
})

In [ ]:
train_ds = emotions["train"]
train_ds

In [ ]:
train_ds[0]

**Preparación de los datos** Los datos se pasan por el mismo tokenizer que se uso para el modelo de base que se utilizará. HuggingFace tiene un método "AutoTokenizer" para esto (simplemente determina cual es el tokenizer para ese modelo y lo aplica)

In [ ]:
from transformers import AutoTokenizer

model_ckpt='distilbert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

Finalmente implementamos una función para aplicar el tokenizer a todo el conjunto de datos

In [ ]:
def tokenize(batch):
  return tokenizer(batch['text'], padding=True,truncation=True)


In [ ]:
emotions_encoded = emotions.map(tokenize,batched=True,batch_size=None)

 **Fine-tuning de un modelo**

 En primer lugar se levanta el modelo a ajustar y luego se procede a entrenarlo.

In [ ]:
from transformers import AutoModelForSequenceClassification
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_labels = 6

model = (AutoModelForSequenceClassification.from_pretrained(model_ckpt,num_labels = num_labels).to(device))

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  f1 = f1_score(labels,preds,average="weighted")
  acc = accuracy_score(labels,preds)
  return {"accuracy": acc, "f1": f1}

In [ ]:
from transformers import Trainer, TrainingArguments

batch_size = 32
logging_steps = len(emotions_encoded["train"])
model_name = f"{model_ckpt}-finetuned-emotions"
training_args = TrainingArguments(output_dir = model_name,
                                  num_train_epochs = 2,
                                  learning_rate= 2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=False,
                                  log_level="error")

In [ ]:
trainer = Trainer(model=model,
                  args=training_args,
                  compute_metrics = compute_metrics,
                  train_dataset=emotions_encoded["train"],
                  eval_dataset = emotions_encoded["validation"],
                  tokenizer=tokenizer)

trainer.train()

In [ ]:
preds_output = trainer.predict(emotions_encoded["test"])

In [ ]:
preds_output.metrics